**Постановка задачи:**  
Во вложении 2 файла, произвольный список операций(операции.xls) и рандомная выгрузка чеков за 2018 год(чеки за 2018.xlsx)  
Соискателю, необходимо на основании этих данных необходимо выдвинуть 2-3 гипотезы и доказать/опровергнуть их.  

Например:  
Не курящие люди чаще берут картошку, чем свеклу.  
С ростом инфляции смещается время, когда люди ходят в магазин  
и тд.

In [1]:
import pandas as pd, numpy as np
import os, xlrd

#Visualization
import plotly
import plotly.plotly as py
from plotly import tools
import plotly.graph_objs as go
import plotly.figure_factory as ff

from scipy import stats

Посмотрим, из чего состоят эксель файлы

In [2]:
excel_files = [f for f in os.listdir() if 'xls' in f]
print('Found excel files in current folder: %s\n'%excel_files)

for file in excel_files:
    sheet_names = xlrd.open_workbook(file, on_demand=True).sheet_names()
    print('%i sheets in %s: %s'%(len(sheet_names), file, sheet_names))

Found excel files in current folder: ['операции.xls', 'чеки за 2018.xlsx']

1 sheets in операции.xls: ['Sheet 1']
2 sheets in чеки за 2018.xlsx: ['чеки ', 'товарные позиции']


Считаем данные и посмотрим на них

In [3]:
ops = pd.read_excel('операции.xls', sheet_name='Sheet 1')
cheq = pd.read_excel('чеки за 2018.xlsx', sheet_name='чеки ')
goods = pd.read_excel('чеки за 2018.xlsx', sheet_name='товарные позиции')

print('ops shape: %s\ncolumns:\n%s\n'%(ops.shape, ops.columns))
print('cheq shape: %s\ncolumns:\n%s\n'%(cheq.shape, cheq.columns))
print('goods shape: %s\ncolumns:\n%s'%(goods.shape, goods.columns))

ops shape: (10000, 9)
columns:
Index(['N21AP12', 'Код идентифицирующий категорию (бизнес-код) точки сервиса.',
       'Сумма операции в валюте карты',
       'Код валюты, в которой выражена сумма транзакции/операции.',
       'Сумма транзакции/операции в валюте транзакции/операции.',
       'Уникальный для каждого Эквайера код, идентифицирующий терминал в точке сервиса,',
       'Код авторизации. Код, присвоенный Участником, разрешившим совершение операции.',
       'Локальные дата и время совершения транзакции.',
       'Параметры пункта обслуживания (название и город).'],
      dtype='object')

cheq shape: (1200, 21)
columns:
Index(['id', 'amount', 'discount_amount', 'discounted_amount',
       'shop_description', 'updated_at', 'points_earned', 'user_id',
       'shop_inn', 'source', 'payment_type', 'cur_id', 'is_cashback_processed',
       'is_cashback_approved', 'date', 'time', 'created', 'bound_to_trans_at',
       'user_added_at', 'approved_by_id', 'received_at'],
      dtype='ob

**Наблюдения**:  
* Датасет с операциями содержит только суммы, коды, время и место операций.
* Датасет с чеками содержит информацию по чеку целиком, включая торговую точку, тип и время получения чека, кэшбек и различные стадии дальнейшей обработки чека
* Товары содержат описания товаров и их количество в каждом чеке + идентификатор пользователя

**Что делаем дальше:**
Логично объединить чеки и товары по id чека и посмотреть на некоторые пересечения в данных

In [4]:
print("%i из %i чеков из goods отсутствуют в cheq"%
      (len([i for i in goods.receipt_id.unique() if i not in cheq.id.unique()]), len(goods.receipt_id.unique())))
print("%i из %i чеков из cheq отсутствуют в goods"%
      (len([i for i in cheq.id.unique() if i not in goods.receipt_id.unique()]), len(cheq.id.unique())))
      
      
comb = (cheq.drop(['updated_at','user_id','created','bound_to_trans_at',
                   'user_added_at','approved_by_id','received_at'], axis=1)
        .merge(goods.drop(['id','updated_at','user_id'], axis=1), 
               left_on='id', right_on='receipt_id')
        .drop('id', axis=1))

1 из 1201 чеков из goods отсутствуют в cheq
0 из 1200 чеков из cheq отсутствуют в goods


In [5]:
print('Объединённый датасет:')
comb.head().T

Объединённый датасет:


,0,1,2,3,4
amount_x,366,500,400,429.37,583
discount_amount,0,0,0,0,0
discounted_amount,366,500,400,429.37,583
shop_description,РОСНЕФТЬ,РОСНЕФТЬ,РОСНЕФТЬ,РОСНЕФТЬ,РОСНЕФТЬ
points_earned,5,6.5,5.5,8.48,5
shop_inn,NaN,NaN,NaN,NaN,NaN
source,1,1,1,1,1
payment_type,NaN,NaN,NaN,NaN,NaN
cur_id,1,1,1,1,1
is_cashback_processed,t,t,t,t,t


In [6]:
print('Процент пропущенных данных по колонкам:')
round(comb.isna().sum()/len(comb)*100,1)

Процент пропущенных данных по колонкам:


amount_x                  0.0
discount_amount           0.0
discounted_amount         0.0
shop_description         14.5
points_earned            59.2
shop_inn                 41.9
source                    0.0
payment_type             43.1
cur_id                    0.0
is_cashback_processed     0.0
is_cashback_approved      0.0
date                      0.0
time                      2.2
amount_y                  0.0
description               0.0
quantity                  0.0
rate                      0.0
receipt_id                0.0
dtype: float64

In [7]:
comb.points_earned.describe()

count    2668.000000
mean      185.437121
std       367.366060
min      -486.780000
25%        21.000000
50%        82.000000
75%       182.000000
max      3000.000000
Name: points_earned, dtype: float64

In [8]:
comb.shop_inn.describe()

count    3.800000e+03
mean     3.338015e+10
std      1.321606e+11
min      2.580138e+08
25%      4.629045e+09
50%      7.707234e+09
75%      7.825706e+09
max      9.102220e+11
Name: shop_inn, dtype: float64

In [9]:
comb.payment_type.value_counts(dropna=False)

 2.0    2883
NaN     2819
 1.0     844
Name: payment_type, dtype: int64

**Наблюдения:**  
Много пропущенных значений в колонках points_earned, shop_inn, payment_type. С первой вроде всё понятно: людим огут расплачиваться баллами и накапливать баллы. Вторая - просто характеристика продавца, а вот третья - не понятно что обозначает: возможно делит на оплату картой, наличными и телефоном или же виза, мастер кард и всё остальное.

**Что делаем:** не вижу необходимости оставлять ИНН для нашего задания. В payment_type заменим NA на 0.

In [10]:
comb.drop('shop_inn', axis=1, inplace=True)
comb.payment_type.fillna(0.0, inplace=True)

Раз уж мы начали работать с этим показателем, посмотрим, как распределяются суммы по различным типам оплаты.

In [12]:
temp = comb.loc[comb.discounted_amount<3000,['receipt_id','payment_type','discounted_amount']].drop_duplicates()

data = [go.Box(y = temp[temp.payment_type==ptype].discounted_amount, name=ptype) for ptype in temp.payment_type.unique()]
layout = go.Layout(title='Boxplots receipt ammounts by payment type (receipt sum under 3k)')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='box_ptype')

C:\Users\moyseand\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



Выдвинем и проверим гипотезу о разности средних размеров (в деньгах) покупок. Запишем гипотезу в терминах статистики:  
**H0: Средняя сумма чека у оплаты типа 0 и оплаты типа 1 не отличается на уровне значимости 95%**  
**H1: На уровне значимости 95% средний чек у оплаты типа 0 выше, чем у оплаты типа 1**  
Таким образом, мы проверяем односторонню гипотезу о равности средних.

Посмотрим на гистограммы распределений.

In [14]:
temp = comb.loc[(comb.discounted_amount<3000)&(comb.payment_type<2),
                ['payment_type','discounted_amount']].drop_duplicates()

hist_data = [temp[temp.payment_type==ptype].discounted_amount
             for ptype in temp.payment_type.unique()]

group_labels = list(temp.payment_type.unique())
colors = ['#333F44', '#94F3E4']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)
fig['layout'].update(title='Receipt ammount by payment type (<3k purchases)')
py.iplot(fig, filename='hist_ptype')

Видим, что распределения едва ли похожи на нормальные. Проверим это тестом Шапиро-Уилкинсона:

In [21]:
payment_0 = temp[temp.payment_type==0.0].discounted_amount.values
payment_1 = temp[temp.payment_type==1.0].discounted_amount.values

print('payment_0 is normally distributed?...',stats.shapiro(payment_0)[1]>0.05)
print('payment_1 is normally distributed?...',stats.shapiro(payment_1)[1]>0.05)

payment_0 is normally distributed?... False
payment_1 is normally distributed?... False


Применим преобразование Бокса-Кокса для нормализации данных и ещё раз посмотрим на гистограммы.

In [16]:
hist_data = [stats.boxcox(temp[temp.payment_type==ptype].discounted_amount)[0]
             for ptype in temp.payment_type.unique()]

group_labels = list(temp.payment_type.unique())
colors = ['#333F44', '#94F3E4']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)
fig['layout'].update(title='Receipt ammount by payment type (<3k purchases) NORMALIZED')
py.iplot(fig, filename='hist_ptype_normalized')

Выглядит значительно лучше.  
Простенькая функция для статистической проверки гипотез

In [19]:
def test_hypothesis(array_1, array_2, one_sided=False, sign_level=0.95):
    t, p = stats.ttest_ind(array_1, array_2)
    if one_sided and t>0 and p/2<(1-sign_level):
        print('H0 hypothesis can be rejected at %.2f significance level'%sign_level)
    elif one_sided==False and p<(1-sign_level):
        print('H0 hypothesis can be rejected at %.2f significance level'%sign_level)
    else:
        print('H0 hypothesis can NOT be rejected at %.2f significance level'%sign_level)        

In [22]:
payment_0 = stats.boxcox(payment_0)[0]
payment_1 = stats.boxcox(payment_1)[0]
test_hypothesis(payment_0, payment_1, one_sided=True, sign_level=0.95)

H0 hypothesis can be rejected at 0.95 significance level


**Вывод:** подтверждаем, что средний чек с payment_type = 0 выше, чем с payment_type=1 

#### Посмотрим на другие пересечения

In [23]:
comb.description.value_counts()[:10]

Бензин                             114
Пакет ПЯТЕРОЧКА 65х40см             85
ПЕРЕКРЕСТОК Пакет майка 65х40см     42
3300573 Пакет ПЯТЕРОЧКА 65х40см     26
АИ-92 К5                            26
Бананы 1кг                          20
БЕЗ НАИМЕНОВАНИЯ                    18
Бананы                      1кг     15
Пакет-майка                         12
Лук репчатый 1кг                    10
Name: description, dtype: int64

Много покупок бензина. Посмотрим, где..

In [24]:
comb[comb.description=='Бензин'].shop_description.value_counts(dropna=False)

РОСНЕФТЬ            72
ТНК                 20
Банк ВБРР" (АО)"    19
Extrim Drive         2
Инстамарт            1
Name: shop_description, dtype: int64

Интересно посмотреть, как соотносится стоимость покупки бензина у различных продавцов.

In [25]:
#убедимся, что количество всегда=1
comb[comb.description=='Бензин'].quantity.describe()

count    114.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: quantity, dtype: float64

In [26]:
temp = comb.loc[(comb.description=='Бензин')&(comb.rate<10000),['shop_description','rate']].drop_duplicates()

data = [go.Box(y = temp[temp.shop_description==shopd].rate, name=shopd) for shopd in temp.shop_description.unique()]
layout = go.Layout(title='Boxplots gazoline purchase rates by seller (rates <10000)')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='box_gazoline_price_by_seller')

Выдвинем и проверим гипотезу о разности средних размеров (в деньгах) покупок бензина у РОСНЕФТЬ и Банк ВБРР. Запишем гипотезу в терминах статистики:  
**H0: Средняя сумма оплаты бензина у РОСНЕФТЬ и Банк ВБРР не отличается на уровне значимости 95%**  
**H1: На уровне значимости 95% средняя сумма оплаты бензина у Банк ВБРР  выше, чем у РОСНЕФТЬ**  
Таким образом, мы проверяем односторонню гипотезу о равности средних.

Посмотрим на гистограммы распределений.

In [27]:
temp1 = temp[temp.shop_description.isin(['РОСНЕФТЬ','Банк ВБРР" (АО)"'])]

hist_data = [temp1[temp1.shop_description==shop].rate for shop in temp1.shop_description.unique()]

group_labels = list(temp1.shop_description.unique())
colors = ['green', 'orange']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)
fig['layout'].update(title='Gazoline Rate by seller (<10k purchases)')
py.iplot(fig, filename='hist_gaz_price_by_seller')

Опять проверим на нормальность.

In [28]:
rneft = temp1[temp1.shop_description=='РОСНЕФТЬ'].rate.values
vbrr = temp1[temp1.shop_description=='Банк ВБРР" (АО)"'].rate.values

print('rneft is normally distributed?...',stats.shapiro(rneft)[1]>0.05)
print('vbrr is normally distributed?...',stats.shapiro(vbrr)[1]>0.05)

rneft is normally distributed?... False
vbrr is normally distributed?... False


Очевидно (и статистически подтверждено), что они распределения не нормальные. Преобразуем и посмотрим на гистограммы ещё раз.

In [29]:
temp1 = temp[temp.shop_description.isin(['РОСНЕФТЬ','Банк ВБРР" (АО)"'])]

hist_data = [stats.boxcox(temp1[temp1.shop_description==shop].rate, lmbda=0) for shop in temp1.shop_description.unique()]

group_labels = list(temp1.shop_description.unique())
colors = ['green', 'orange']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)
fig['layout'].update(title='Gazoline Rate by seller (<10k purchases) NORMALIZED')
py.iplot(fig, filename='hist_gaz_price_by_seller_normalized')

Не идеально, но хотя бы приблизили к нормальному

In [30]:
rneft = stats.boxcox(rneft, lmbda=0)
vbrr = stats.boxcox(vbrr, lmbda=0)
test_hypothesis(vbrr, rneft, one_sided=True, sign_level=0.95)

H0 hypothesis can be rejected at 0.95 significance level


**Вывод:** подтверждаем, средняя сумма покупки бензина у "Банк ВБРР" выше, чем у "РОСНЕФТЬ"

#### P.S. Подобным образом можно придумать и проверить множество гипотез, в зависимости от бизнес задач.